<a href="https://colab.research.google.com/github/HanqiLouis/GFET-Characterization/blob/main/GFET_Characterization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run this cell to access your personal Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# @title Functions
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


def plot_site_runs(directory_paths, folders_names, site_number, runs_per_site=3, dual_sweep=False, save_output=None, display=False):
    """
    Plots the runs for a specific site for multiple folders and optionally saves the plot.

    Parameters:
    - directory_paths: list of str, paths to the folders containing Excel files.
    - folders_names: list of str, names of the folders for labeling.
    - site_number: int, the site to plot (1-indexed).
    - runs_per_site: int, number of runs per site (default: 3).
    - dual_sweep: bool, if the measurement mode was dual, set dual_sweep to True to only plot the forward part.
    - save_output: str or None, path to the folder where the plot should be saved (default: None).
    """
    # Helper function to get files for a site from a folder
    def get_site_files(directory_path):
        start_index = (site_number - 1) * runs_per_site
        end_index = start_index + runs_per_site
        excel_files = sorted(glob.glob(os.path.join(directory_path, '*.xlsx')))
        return excel_files[start_index:end_index]

    plt.figure(figsize=(8, 6))

    # Iterate through all provided folders
    for folder_index, directory_path in enumerate(directory_paths, start=1):
        site_files = get_site_files(directory_path)

        if not site_files:
            print(f"No files found for Site {site_number} in folder {folder_index}.")
            continue

        # Plot runs from the current folder
        for run_index, file in enumerate(site_files, start=1):
            df = pd.read_excel(file)
            if dual_sweep:
                total_rows = df.shape[0]
                df_subset = df.iloc[0:int(total_rows / 2)]
                Vg = df_subset['VG']
                Id = df_subset['ID']
            else:
                Vg = df['VG']
                Id = df['ID']
            plt.plot(
                1000 * Vg,
                1000000 * Id,
                marker='.',
                markersize=1,
                linestyle='-',
                label=f'{folders_names[folder_index-1]} Run {run_index}',
                linewidth=1
            )

    # Add labels, title, and legend
    plt.xlabel('VG [mV]')
    plt.ylabel('ID [uA]')
    plt.title(f'Site {site_number}')
    plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1))
    plt.tight_layout()
    plt.grid()

    # Save the plot if save_output is specified
    if save_output is not None:
        os.makedirs(save_output, exist_ok=True)  # Create the directory if it doesn't exist
        plot_path = os.path.join(save_output, f'site_{site_number}_plot.png')
        plt.savefig(plot_path, dpi=300)
        print(f"Plot saved to {plot_path}")

    # Display the plot if display is True
    if display:
      plt.show()
    else:
      plt.close()


def plot_average_runs(groups_of_folders, names_of_runs, site_number, runs_per_site_list, dual_sweep_list, save_output=None, display=False):
    """
    Plots the average runs for specific groups of folders on the same plot and optionally saves the plot.

    Parameters:
    - groups_of_folders: list of list of str, each sublist contains paths to folders representing a group.
    - names_of_runs: list of str, names of the groups for labeling.
    - site_number: int, the site to plot (1-indexed).
    - runs_per_site_list: list of int, each value represents the runs per site for the corresponding group.
    - dual_sweep_list: list of bool, each value represents whether dual sweep is enabled for the corresponding group.
    - save_output: str or None, path to the folder where the plot should be saved (default: None).
    """

    # Iterate through each group of folders
    for group_index, (folder_group, runs_per_site, dual_sweep) in enumerate(zip(groups_of_folders, runs_per_site_list, dual_sweep_list)):
        all_runs = []  # To store all runs for averaging in the current group

        # Helper function to get files for a site from a folder
        def get_site_files(directory_path):
            start_index = (site_number - 1) * runs_per_site
            end_index = start_index + runs_per_site
            excel_files = sorted(glob.glob(os.path.join(directory_path, '*.xlsx')))
            return excel_files[start_index:end_index]

        # Iterate through folders in the current group
        for folder_index, directory_path in enumerate(folder_group):
            site_files = get_site_files(directory_path)

            if not site_files:
                print(f"No files found for Site {site_number} in folder {directory_path}.")
                continue

            # Collect runs from the current folder
            for file in site_files:
                df = pd.read_excel(file)

                # Apply dual sweep logic for this specific group
                if dual_sweep:
                    total_rows = df.shape[0]
                    df_subset = df.iloc[0:int(total_rows / 2)]
                    Vg = df_subset['VG']
                    Id = df_subset['ID']
                else:
                    Vg = df['VG']
                    Id = df['ID']

                all_runs.append((1000 * Vg, 1000000 * Id))  # Convert units

        # Compute and plot the average run for the current group
        if all_runs:
            # Interpolate to align all runs to the same VG points
            common_Vg = np.linspace(
                min(run[0].min() for run in all_runs),
                max(run[0].max() for run in all_runs),
                500  # Number of points for interpolation
            )
            interpolated_Id = [np.interp(common_Vg, Vg, Id) for Vg, Id in all_runs]

            avg_Id = np.mean(interpolated_Id, axis=0)

            plt.plot(
                common_Vg,
                avg_Id,
                linestyle='-',
                linewidth=2,
                label=f'{names_of_runs[group_index]}'
            )

    # Add labels, title, and legend
    plt.xlabel('VG [mV]')
    plt.ylabel('ID [uA]')
    plt.title(f'Site {site_number} - Average Runs Comparison')
    plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1))
    plt.tight_layout()
    plt.grid()

    # Save the plot if save_output is specified
    if save_output:
        os.makedirs(save_output, exist_ok=True)  # Create the directory if it doesn't exist
        plot_path = os.path.join(save_output, f'site_{site_number}_average_runs_plot.png')
        plt.savefig(plot_path, dpi=300)
        print(f"Plot saved to {plot_path}")

    if display:
        plt.show()
    else:
        plt.close()

Mounted at /content/drive


In [ ]:
from scipy import stats

def DP_forward(directory_path, site_number, runs_per_site, dual_sweep=False, remove_outliers=False, print_option=True):
    """
    Extracts the forward DP (Dirac Point) Id and Vg for a specific site and checks for potential malfunctions.
    """

    start_index = (site_number - 1) * runs_per_site
    end_index = start_index + runs_per_site
    excel_files = sorted(glob.glob(os.path.join(directory_path, '*.xlsx')))
    site_files = excel_files[start_index:end_index]

    if not site_files:
        print(f"No files found for Site {site_number}.")
        return None

    I, V = [], []

    for file in site_files:
        df = pd.read_excel(file)

        if dual_sweep:
            df = df.iloc[: df.shape[0] // 2]  # First half of data for forward sweep

        # Extract Vg and Id
        Vg = df['VG']
        Id = df['ID']

        # Find the Dirac Point (minimum Id)
        min_Id = Id.min()
        DP_forward_candidates = df.loc[df['ID'] == min_Id, 'VG'].values

        if len(DP_forward_candidates) > 0:
            DP_forward = DP_forward_candidates[0]  # Take first match if multiple exist
        else:
            continue  # Skip this run if no valid DP found

        # Convert to uA and mV
        I.append(1000000 * min_Id)
        V.append(1000 * DP_forward)

    if not I or not V:
        print(f"No valid Dirac Point data found for Site {site_number}.")
        return None

    D = np.column_stack((I, V))

    if print_option:
        print("Raw Data:\n", D)

    if remove_outliers:
        z_scores = stats.zscore(D, axis=0, nan_policy='omit')
        valid_runs = (np.abs(z_scores) < 2).all(axis=1)

        if valid_runs.any():
            D_filtered = D[valid_runs]
            deleted_runs = [i + 1 for i, is_valid in enumerate(valid_runs) if not is_valid]
            print(f"Deleted runs due to outliers: {deleted_runs}")
            print("Filtered Data:\n", D_filtered)
            return D_filtered
        else:
            print("All runs were classified as outliers. Site may be malfunctioning.")
            return None

    return D



def compute_full_refiling(folder_path, total_site_nb, runs_per_site, dual_sweep=False, remove_outliers=False):
    """
    Extract the Id and Vg at forward DP for all sites of a refiling.

    Returns:
    - combined_data: numpy array of shape (num_sites, num_runs, 2).
    """

    Data = []
    for site in range(1, total_site_nb+1):
        D = DP_forward(folder_path, site, runs_per_site, dual_sweep, remove_outliers, print_option=False)
        if D is not None:  # Ensure valid data
            Data.append(D)

    if Data:  # Ensure list is not empty before stacking
        combined_data = np.stack(Data, axis=0)
        return combined_data
    else:
        print("No valid data found across all sites.")
        return None



def filter_malfunctioning_sites(all_refilings, malfunctioning_sites):
    """
    Computes the mean Vg and Id across multiple matrices and removes malfunctioning sites.
    """

    # Concatenate all matrices along axis=1 (merge runs)
    all_refilings_conc = np.concatenate(all_refilings, axis=1)

    # Extract Vg and Id values
    Vg = all_refilings_conc[:, :, 1]  # Index 1 for Vg
    Id = all_refilings_conc[:, :, 0]  # Index 0 for Id

    # Compute mean along runs
    mean_Vg = np.mean(Vg, axis=1)
    mean_Id = np.mean(Id, axis=1)

    if not malfunctioning_sites:
        return mean_Vg, mean_Id  # No need to remove anything

    # Convert 1-based indices to 0-based and filter valid indices
    max_index = len(mean_Vg)
    index_to_remove = [i - 1 for i in malfunctioning_sites if 0 <= (i - 1) < max_index]

    if index_to_remove:
        filtered_mean_Vg = np.delete(mean_Vg, index_to_remove)
        filtered_mean_Id = np.delete(mean_Id, index_to_remove)
        return filtered_mean_Vg, filtered_mean_Id
    else:
        print("No valid malfunctioning sites found in range.")
        return mean_Vg, mean_Id



def plot_boxplot_mean_functioning_sites(filtered_Vg_matrix, label, ID=False):
    """
    Plots a boxplot and scatter plot for a single dataset of VG or ID.
    """
    fig, ax = plt.subplots(figsize=(6, 4))

    ax.boxplot(filtered_Vg_matrix, notch=True, patch_artist=True, boxprops=dict(facecolor="lightblue"))

    # Scatter points
    x_positions = np.random.normal(1, 0.02, size=len(filtered_Vg_matrix))
    ax.scatter(x_positions, filtered_Vg_matrix, color="darkblue", alpha=0.6)

    # Labels
    ax.set_title("Dirac Points of Functioning Sites", fontsize=14)
    ax.set_ylabel("VG [mV]" if not ID else "ID [uA]")

    ax.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()
    plt.show()



def plot_distribution(data, labels, title, ylabel, save_output=None, file_name='boxplot'):
    fig, ax = plt.subplots(figsize=(6, 5))
    ax.boxplot(data, notch=True, patch_artist=True, boxprops=dict(facecolor="lightblue"))

    for i, matrix in enumerate(data, start=1):
        x_positions = np.random.normal(i, 0.05, size=len(matrix))
        ax.scatter(x_positions, matrix, alpha=0.6)

    ax.set_xticks(range(1, len(labels) + 1))
    ax.set_xticklabels(labels, fontsize=12)
    ax.set_title(title, fontsize=14)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()

    # Save the plot if save_output is specified
    if save_output is not None:
        os.makedirs(save_output, exist_ok=True)  # Create the directory if it doesn't exist
        plot_path = os.path.join(save_output, f'{file_name}.png')
        plt.savefig(plot_path, dpi=300)
        print(f"Plot saved to {plot_path}")

    plt.show()


def plot_vg_distribution(data, labels, title="Dirac Points of Functioning Sites", save_output=None, file_name='boxplot'):
    plot_distribution(data, labels, title, "VG [mV]", save_output, file_name)

def plot_id_distribution(data, labels, title="Current Distribution of Functioning Sites", save_output=None, file_name='boxplot'):
    plot_distribution(data, labels, title, "ID [uA]", save_output, file_name)


In [12]:
# @title Plot all Runs
import ipywidgets as widgets
from IPython.display import display, clear_output

### ------- Global Variables -------
folder_path_widgets = []  # List of text fields for folder paths
folder_name_widgets = []  # List of text fields for folder names
output = widgets.Output()  # Output display for results

### ------- Folder Management Functions -------
def add_folder(_=None):
    """ Adds a new folder input set (path & name) dynamically """
    index = len(folder_path_widgets) + 1

    path_widget = widgets.Text(
        description=f'Path to Measurement Folder {index}:',
        placeholder=f'Enter path for Measurement Folder {index}',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='750px')
    )

    name_widget = widgets.Text(
        description=f'Measurement {index} Name:',
        placeholder=f'Enter name for Measurement {index}',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='350px')
    )

    folder_path_widgets.append(path_widget)
    folder_name_widgets.append(name_widget)
    update_ui()

def remove_folder(_):
    """ Removes the last added folder input set, ensuring at least one remains """
    if len(folder_path_widgets) > 1:
        folder_path_widgets.pop()
        folder_name_widgets.pop()
        update_ui()

### ------- Buttons for Adding & Removing Folders -------
add_button = widgets.Button(description="+", button_style="success")
remove_button = widgets.Button(description="-", button_style="danger")

add_button.on_click(add_folder)
remove_button.on_click(remove_folder)

### ------- Site & Run Fields -------
site_field = widgets.BoundedIntText(
    value=0, min=0, description='Number of sites:', style={'description_width': 'initial'}
)

run_field = widgets.BoundedIntText(
    value=0, min=0, description='Runs per site:', style={'description_width': 'initial'}
)

### ------- Dropdown for Site Selection -------
dropdown = widgets.Dropdown(options=[0], value=0, description='Site to display:', style={'description_width': 'initial'})

def update_dropdown_options(change):
    """ Updates dropdown options based on site_field value """
    dropdown.options = [i for i in range(1, change['new'] + 1)]

site_field.observe(update_dropdown_options, names='value')

### ------- Checkbox for Dual Sweep -------
dual_sweep_checkbox = widgets.Checkbox(value=False, description='Dual sweep', style={'description_width': 'initial'})

### ------- Save Functionality -------
save_checkbox = widgets.Checkbox(value=False, description='Save', style={'description_width': 'initial'})

save_directory_field = widgets.Text(
    description='Save Directory:',
    placeholder='Enter directory to save files',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='750px')
)

### ------- Plot Runs Function -------
def plot_runs(_):
    """ Processes input values and calls plot function """
    folder_path = []
    folder_name = []

    with output:
        clear_output()

        if site_field.value == 0:
            print("Please enter the number of sites")
            return
        if run_field.value == 0:
            print("Please enter the number of runs per site")
            return

        # Count the number of provided folders
        j = sum(1 for widget in folder_path_widgets if widget.value.strip())

        if j == 0:
            print("No folder provided")
            return

        # Collect paths & names
        for i in range(j):
            folder_path.append(folder_path_widgets[i].value.strip())
            folder_name.append(folder_name_widgets[i].value.strip() or f'Refiling{i+1}')

        # Call the external function plot_site_runs()
        if save_checkbox.value:
            if not save_directory_field.value.strip():
                print("Please enter the save directory")
                return
            else:
                plot_site_runs(
                    folder_path, folder_name, site_number=dropdown.value,
                    runs_per_site=run_field.value, dual_sweep=dual_sweep_checkbox.value,
                    save_output=save_directory_field.value.strip(), display=True
                )
        else:
            plot_site_runs(
                folder_path, folder_name, site_number=dropdown.value,
                runs_per_site=run_field.value, dual_sweep=dual_sweep_checkbox.value,
                save_output=None, display=True
            )

### ------- Save All Plots Function -------
def save_all_plots(_):
    """ Saves all plots to a specified directory """

    folder_path = []
    folder_name = []

    with output:
        clear_output()

        if not save_directory_field.value.strip():
            print("Please enter the save directory")
            return

        j = sum(1 for widget in folder_path_widgets if widget.value.strip())

        if j == 0:
            print("No folder provided")
            return

        if site_field.value == 0:
            print("Please enter the number of sites")
            return
        if run_field.value == 0:
            print("Please enter the number of runs per site")
            return
                # Collect paths & names
        for i in range(j):
            folder_path.append(folder_path_widgets[i].value.strip())
            folder_name.append(folder_name_widgets[i].value.strip() or f'Refiling{i+1}')

        for i in range(site_field.value):
            plot_site_runs(
                folder_path, folder_name, site_number=i+1,
                runs_per_site=run_field.value, dual_sweep=dual_sweep_checkbox.value,
                save_output=save_directory_field.value.strip(), display=False
            )

        print("All plots saved in:", save_directory_field.value.strip())

### ------- Buttons for Plotting & Saving -------
plot_button = widgets.Button(description='Plot', button_style='warning')
plot_button.on_click(plot_runs)

save_all_button = widgets.Button(description='Save all plots', button_style='primary')
save_all_button.on_click(save_all_plots)

### ------- UI Update Function (Defined at the End) -------
def update_ui():
    """ Clears and updates the UI with the latest state of widgets """
    clear_output(wait=True)

    # Display all dynamic folder fields
    for path, name in zip(folder_path_widgets, folder_name_widgets):
        display(path)
        display(name)

    # Display action buttons
    display(widgets.HBox([add_button, remove_button]))

    # Display the rest of the widgets
    display(site_field, run_field)
    display(dropdown)
    display(dual_sweep_checkbox, save_checkbox)
    display(save_directory_field)
    display(plot_button, output)
    display(save_all_button)

# Initialize the first folder field AFTER defining everything
add_folder(None)
update_ui()


Text(value='', description='Path to Measurement Folder 1:', layout=Layout(width='750px'), placeholder='Enter p…

Text(value='', description='Measurement 1:', layout=Layout(width='350px'), placeholder='Enter name for Measure…

BoundedIntText(value=0, description='Number of sites:', style=DescriptionStyle(description_width='initial'))

BoundedIntText(value=0, description='Runs per site:', style=DescriptionStyle(description_width='initial'))

Dropdown(description='Site to display:', options=(0,), style=DescriptionStyle(description_width='initial'), va…

...

Checkbox(value=False, description='Save', style=DescriptionStyle(description_width='initial'))

Text(value='', description='Save Directory:', layout=Layout(width='750px'), placeholder='Enter directory to sa…

Button(button_style='warning', description='Plot', style=ButtonStyle())

Output()

Button(button_style='primary', description='Save all plots', style=ButtonStyle())

In [ ]:
# @title Plot Average Runs
import ipywidgets as widgets
from IPython.display import display, clear_output

### ------- Global Variables -------
groups = []  # List of groups containing folders
output = widgets.Output()  # Output area for displaying information

### ------- Group Class -------
class Group:
    def __init__(self, index):
        """ Initialize a group with name, runs per site, dual sweep checkbox, and folders """
        self.index = index

        # Group name field
        self.group_name = widgets.Text(
            description=f'Measurement Group {index} Name:',
            placeholder=f'Enter name for Measurement Group {index}',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='750px')
        )

        # Runs per site (positive integer)
        self.runs_per_site = widgets.BoundedIntText(
            value=0, min=0,
            description="Runs per site:",
            style={'description_width': 'initial'}
        )

        # Checkbox for dual sweep
        self.dual_sweep = widgets.Checkbox(
            value=False,
            description="Dual sweep"
        )

        # Folder storage
        self.folders = []
        self.add_folder()  # Ensure at least one folder initially

        # Buttons for adding/removing folders
        self.add_folder_button = widgets.Button(description="+ Folder", button_style="success")
        self.remove_folder_button = widgets.Button(description="- Folder", button_style="danger")

        self.add_folder_button.on_click(self.add_folder)
        self.remove_folder_button.on_click(self.remove_folder)

    def add_folder(self, _=None):
        """ Add a new folder to this group """
        folder_widget = widgets.Text(
            description=f'Folder {len(self.folders) + 1}:',
            placeholder=f'Enter path for Folder {len(self.folders) + 1}',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='750px')
        )
        self.folders.append(folder_widget)
        update_ui()

    def remove_folder(self, _=None):
        """ Remove the last folder, ensuring at least one remains """
        if len(self.folders) > 1:
            self.folders.pop()
            update_ui()

### ------- UI Update Function -------
def update_ui():
    """ Clears and updates the UI with the latest state of widgets """
    clear_output(wait=True)

    # Display all groups and their folders
    for group in groups:
        display(group.group_name)

        for folder in group.folders:
            display(folder)

        display(group.runs_per_site)
        display(group.dual_sweep)
        display(widgets.HBox([group.add_folder_button, group.remove_folder_button]))
        display(widgets.HTML('<hr>'))  # Separator

    # Display global controls
    display(widgets.HBox([add_group_button, remove_group_button]))
    display(nb_sites)
    display(site_to_display)
    display(save_checkbox)
    display(save_directory_field)
    display(plot_button, output)
    display(save_all_button)

### ------- Group Management Functions -------
def add_group(_=None):
    """ Add a new group with an initial folder """
    groups.append(Group(len(groups) + 1))
    update_ui()

def remove_group(_=None):
    """ Remove the last added group, ensuring at least one remains """
    if len(groups) > 1:
        groups.pop()
        update_ui()

### ------- Plot & Save Functions -------
def validate_inputs():
    """ Check if required fields are filled before plotting """
    with output:
        output.clear_output()

        for group in groups:
            if group.runs_per_site.value == 0:
                print(f"{group.group_name.value or f'Measurement {group.index}'}: Please enter the number of runs per site")
                return False

            for idx, folder in enumerate(group.folders, start=1):
                if not folder.value.strip():
                    print(f"{group.group_name.value or f'Measurement {group.index}'}: No input provided for Folder {idx}")
                    return False

        if nb_sites.value == 0:
            print("Please enter the number of sites")
            return False

        if site_to_display.value == 0:
            print("Please enter the site to display")
            return False

        return True

def collect_group_data():
    """ Collect and structure group data for plotting """
    groups_list, groups_names, runs_per_site_list, dual_sweep_list = [], [], [], []

    for group in groups:
        groups_names.append(group.group_name.value or f"Measurement {group.index}")
        runs_per_site_list.append(group.runs_per_site.value)
        dual_sweep_list.append(group.dual_sweep.value)
        groups_list.append([folder.value.strip() for folder in group.folders])

    return groups_list, groups_names, runs_per_site_list, dual_sweep_list

def plot_average(_=None):
    """ Process inputs and call plot function """
    with output:
        output.clear_output()

        if not validate_inputs():
            return

        groups_list, groups_names, runs_per_site_list, dual_sweep_list = collect_group_data()

        if save_checkbox.value and not save_directory_field.value.strip():
            print("Please enter the save directory")
            return

        plot_average_runs(groups_list, groups_names, site_to_display.value, runs_per_site_list, dual_sweep_list,
                          save_output=save_directory_field.value.strip() if save_checkbox.value else None, display=True)

def save_all_average_plots(_=None):
    """ Save all plots for each site """
    if not validate_inputs():
        return

    if not save_directory_field.value.strip():
        print("Please enter the save directory")
        return

    groups_list, groups_names, runs_per_site_list, dual_sweep_list = collect_group_data()

    for i in range(nb_sites.value):
        plot_average_runs(groups_list, groups_names, i + 1, runs_per_site_list, dual_sweep_list,
                          save_output=save_directory_field.value.strip(), display=False)

### ------- Widgets -------
nb_sites = widgets.BoundedIntText(value=0, min=0, description="Number of sites:", style={'description_width': 'initial'})
site_to_display = widgets.BoundedIntText(value=0, min=0, description="Site to display:", style={'description_width': 'initial'})

save_checkbox = widgets.Checkbox(value=False, description='Save', style={'description_width': 'initial'})
save_directory_field = widgets.Text(
    description='Save Directory:',
    placeholder='Enter directory to save files',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='750px')
)

add_group_button = widgets.Button(description="+ Group", button_style="success")
remove_group_button = widgets.Button(description="- Group", button_style="danger")
add_group_button.on_click(add_group)
remove_group_button.on_click(remove_group)

plot_button = widgets.Button(description='Plot', button_style='warning')
plot_button.on_click(plot_average)

save_all_button = widgets.Button(description='Save all plots', button_style='primary')
save_all_button.on_click(save_all_average_plots)

### ------- Initialize UI -------
add_group(None)
update_ui()


In [ ]:
# @title Boxplots
import ipywidgets as widgets
from IPython.display import display, clear_output

### ------- Global Variables -------
groups = []  # List of groups containing folders
output = widgets.Output()  # Output display for results

### ------- Group Class -------
class Group:
    def __init__(self, index):
        """ Initialize a measurement group with all required fields. """
        self.index = index

        # Group name
        self.group_name = widgets.Text(
            description=f'Measurement Group {index} Name:',
            placeholder=f'Enter name for Group {index}',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='750px')
        )

        # Number of sites
        self.nb_sites = widgets.BoundedIntText(
            value=0, min=0,
            description="Number of sites:",
            style={'description_width': 'initial'}
        )

        # Runs per site
        self.runs_per_site = widgets.BoundedIntText(
            value=0, min=0,
            description="Runs per site:",
            style={'description_width': 'initial'}
        )

        # Checkboxes
        self.dual_sweep = widgets.Checkbox(value=False, description="Dual sweep")
        self.remove_outliers = widgets.Checkbox(value=False, description="Remove outliers")

        # Malfunctioning sites input
        self.malfunctioning_sites = widgets.Text(
            description=f'Malfunctioning sites:',
            placeholder=f'Enter site numbers (comma-separated)',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='500px')
        )

        # Folder storage
        self.folders = []
        self.add_folder()  # Ensure at least one folder initially

        # Buttons for folder management
        self.add_folder_button = widgets.Button(description="+ Folder", button_style="success")
        self.remove_folder_button = widgets.Button(description="- Folder", button_style="danger")

        self.add_folder_button.on_click(self.add_folder)
        self.remove_folder_button.on_click(self.remove_folder)

    def add_folder(self, _=None):
        """ Adds a new folder input field to the group. """
        folder_widget = widgets.Text(
            description=f'Folder {len(self.folders) + 1}:',
            placeholder=f'Enter folder path',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='750px')
        )
        self.folders.append(folder_widget)
        update_ui()

    def remove_folder(self, _=None):
        """ Removes the last folder, ensuring at least one remains. """
        if len(self.folders) > 1:
            self.folders.pop()
            update_ui()

### ------- UI Update Function -------
def update_ui():
    """ Clears and updates the UI dynamically. """
    clear_output(wait=True)

    for group in groups:
        display(group.group_name)

        for folder in group.folders:
            display(folder)

        display(widgets.HBox([group.nb_sites, group.runs_per_site]))
        display(widgets.HBox([group.dual_sweep, group.remove_outliers]))
        display(group.malfunctioning_sites)
        display(widgets.HBox([group.add_folder_button, group.remove_folder_button]))
        display(widgets.HTML('<hr>'))  # Separator

    # Display global controls
    display(widgets.HBox([add_group_button, remove_group_button]))
    display(mode)
    display(save_checkbox)
    display(save_directory_field, save_as)
    display(plot_button, output)

### ------- Group Management Functions -------
def add_group(_=None):
    """ Adds a new group with an initial folder. """
    groups.append(Group(len(groups) + 1))
    update_ui()

def remove_group(_=None):
    """ Removes the last group, ensuring at least one remains. """
    if len(groups) > 1:
        groups.pop()
        update_ui()

### ------- Input Validation -------
def validate_inputs():
    """ Validates required fields before processing data. """
    with output:
        output.clear_output()

        for group in groups:
            if group.nb_sites.value == 0:
                print(f"{group.group_name.value or f'Group {group.index}'}: Please enter the number of sites")
                return False
            if group.runs_per_site.value == 0:
                print(f"{group.group_name.value or f'Group {group.index}'}: Please enter the number of runs per site")
                return False

            for idx, folder in enumerate(group.folders, start=1):
                if not folder.value.strip():
                    print(f"{group.group_name.value or f'Group {group.index}'}: Folder {idx} path is missing")
                    return False

        return True

### ------- Data Processing -------
def collect_group_data():
    """ Collects input data from UI into structured lists. """
    groups_list, groups_names, runs_per_site_list, dual_sweep_list = [], [], [], []

    for group in groups:
        groups_names.append(group.group_name.value or f"Group {group.index}")
        runs_per_site_list.append(group.runs_per_site.value)
        dual_sweep_list.append(group.dual_sweep.value)
        groups_list.append([folder.value.strip() for folder in group.folders])

    return groups_list, groups_names, runs_per_site_list, dual_sweep_list

### ------- Plotting Functions -------
def plot_boxplot(_=None):
    """ Processes data and generates a boxplot based on user input. """
    with output:
        output.clear_output()

        if not validate_inputs():
            return

        if save_checkbox.value and not save_directory_field.value,strip():
            print("Please enter the save directory")
            return

        Vgs, Ids = [], []
        groups_list, groups_names, runs_per_site_list, dual_sweep_list = collect_group_data()

        for group in groups:
            measures = []
            for folder in group.folders:
                R = compute_full_refiling(
                    folder.value.strip(),
                    group.nb_sites.value,
                    group.runs_per_site.value,
                    group.dual_sweep.value,
                    group.remove_outliers.value
                )
                measures.append(R)

            malfunctioning_sites = [int(x.strip()) for x in group.malfunctioning_sites.value.split(",") if x.strip().isdigit()]
            filtered_Vg_matrix, filtered_Id_matrix = filter_malfunctioning_sites(measures, malfunctioning_sites)
            Vgs.append(filtered_Vg_matrix)
            Ids.append(filtered_Id_matrix)

        if mode.value == "Id":
            plot_id_distribution(Ids, groups_names, save_output=save_directory_field.value.strip() if save_checkbox.value else None,
                                 file_name=save_as.value.strip() if save_as.value else 'boxplot')
        else:
            plot_vg_distribution(Vgs, groups_names, save_output=save_directory_field.value.strip() if save_checkbox.value else None,
                                 file_name=save_as.value.strip() if save_as.value else 'boxplot')

### ------- Widgets -------
mode = widgets.Dropdown(options=["VG", "ID"], value="VG", description="Mode:", style={'description_width': 'initial'})
save_checkbox = widgets.Checkbox(value=False, description='Save', style={'description_width': 'initial'})

save_directory_field = widgets.Text(description='Save Directory:', placeholder='Enter save directory',
                                    style={'description_width': 'initial'}, layout=widgets.Layout(width='750px'))

save_as = widgets.Text(description='Save as:', placeholder='Enter file name',
                       style={'description_width': 'initial'}, layout=widgets.Layout(width='500px'))

add_group_button = widgets.Button(description="+ Group", button_style="success")
remove_group_button = widgets.Button(description="- Group", button_style="danger")
add_group_button.on_click(add_group)
remove_group_button.on_click(remove_group)

plot_button = widgets.Button(description='Plot', button_style='warning')
plot_button.on_click(plot_boxplot)

### ------- Initialize UI -------
add_group(None)
update_ui()